# 数据模块 (Data Module)

## 📖 概述

数据模块提供了一套完整的数据处理解决方案，涵盖从数据加载、预处理、质量标注到导出的全生命周期流程。该模块支持多种运行模式，可以灵活地组合不同的数据处理组件，满足各种数据处理场景的需求。

## 🏗️ 系统架构

### 核心组件介绍

数据模块采用模块化设计，由以下五个核心组件构成：

1. **Load Module (加载模块)**
   - 🌐 支持本地文件和远程数据源（如 HuggingFace Hub）
   - 📊 支持多种数据格式：parquet、jsonl、json 等
   - 🔌 内置数据源适配器：rewardbench、chatmessage、prmbench 等
   - 🎯 支持数据分割和采样限制

2. **Process Module (处理模块)**
   - ⚙️ 可配置的数据处理管道
   - 🔍 内置过滤器：文本长度过滤、对话轮次过滤等
   - 🧹 集成 data-juicer 高级数据清洗操作符
   - 🎨 支持自定义处理器扩展

3. **Annotation Module (标注模块)**
   - 🏷️ 深度集成 Label Studio 标注平台
   - 📋 支持多种预设标注模板
   - 🚀 自动项目创建和配置管理
   - 👥 支持多用户协作标注

4. **Export Module (导出模块)**
   - 💾 多格式数据导出：jsonl、parquet、json
   - ✂️ 智能数据分割（训练集/测试集）
   - 📁 保持原始数据目录结构

5. **Build Module (构建模块)**
   - 🔧 统一的数据管道编排
   - 🌊 模块间数据流自动管理
   - 📝 支持 YAML 配置化构建
   - 🔄 支持管道复用和扩展

## 🚀 运行模式

数据模块支持两种主要的运行方式：**Python 脚本模式**和 **YAML 配置模式**，满足不同用户的使用习惯。

> 💡 **参考示例**: 完整的配置示例请参考 `/examples/data/data_config.yaml`

### 🐍 Python 脚本模式 (data_pipeline.py)

#### 1. 基础数据处理流程
执行完整的数据处理管道：**数据加载 → 数据处理 → 数据导出**

```bash
# 处理 100 条样本数据
python data_pipeline.py --mode basic --limit 100
```

#### 2. 包含标注的完整流程
执行包含人工标注的完整管道：**数据加载 → 数据处理 → 数据标注 → 数据导出**

```bash
# 需要提供 Label Studio API Token
python data_pipeline.py --mode annotation --api-token YOUR_LABEL_STUDIO_TOKEN
```

#### 3. 独立模块测试模式
支持单独测试各个模块的功能：

- **仅数据加载**: `python data_pipeline.py --mode load-only`
- **仅数据处理**: `python data_pipeline.py --mode process-only`  
- **仅数据导出**: `python data_pipeline.py --mode export-only`

#### 4. 标注数据导出模式
从 Label Studio 导出已完成标注的数据：

```bash
python data_pipeline.py --mode export-annotation \
    --api-token YOUR_TOKEN \
    --project-id PROJECT_ID
```

### 📄 YAML 配置模式 (data_from_yaml.py)

通过声明式的 YAML 配置文件运行数据管道，更适合批量处理和生产环境：

```bash
python data_from_yaml.py
```



## ⚙️ 配置文件详解

### YAML 配置文件结构

YAML 配置文件提供了声明式的管道配置方式，支持完整的数据处理流程定义。以下是完整的配置文件结构说明：

```yaml
dataset:
    # 数据集基本信息
    name: rewardbench2                    # 数据集名称
                                          # local 模式: 自定义名称（如 rewardbench2）
                                          # huggingface 模式: HF 数据集名称（如 allenai/reward-bench-2）
    
    # 数据源配置
    configs:
        type: local                       # 数据源类型
                                          # - local: 本地文件系统
                                          # - huggingface: HuggingFace Hub
        source: rewardbench2              # 数据源适配器标识符
                                          # 注意：需确保对应的转换器已注册
        path: /path/to/data.parquet       # 数据文件路径（仅 local 模式有效）
        huggingface_split: train          # 数据分割名称（仅 huggingface 模式有效）
                                          # 可选值: train, test, validation 等
        limit: 2000                       # 样本数量限制（随机采样）
                                          # 用于快速测试或数据预览
    
    # 数据处理器配置（可选）
    processors:
        # 对话轮次过滤器
        - type: filter
          name: conversation_turn_filter
          config:
            min_turns: 1                  # 最小对话轮次
            max_turns: 6                  # 最大对话轮次
        
        # 文本长度过滤器
        - type: filter
          name: text_length_filter
          config:
            min_length: 10                # 最小文本长度（字符数）
            max_length: 1000              # 最大文本长度（字符数）
        
        # data-juicer 高级清洗器
        - type: data_juicer
          name: character_repetition_filter
          config:
            rep_len: 10                   # 重复字符序列长度阈值
            min_ratio: 0.0                # 重复字符比例下限
            max_ratio: 0.5                # 重复字符比例上限
    
    # 标注配置（可选）
    annotation:
        template_name: "rewardbench2"     # 标注模板名称
        project_title: "Reward Bench Evaluation"  # Label Studio 项目标题
        project_description: "Reward model evaluation using reward bench template from yaml"
        server_url: "http://localhost:8080"        # Label Studio 服务器地址
        api_token: "your_api_token_here"          # Label Studio API 令牌
    
    # 导出配置（必选）
    export:
        output_dir: ./examples/data/exports       # 导出目录路径
        formats: ["jsonl"]                        # 导出格式列表
                                                  # 支持: jsonl, parquet, json
        preserve_structure: true                  # 是否保持原始目录结构
        split_ratio: {"train": 0.8, "test": 0.2} # 数据集分割比例
                                                  # 支持多种分割: train/test，不需要划分时直接注释该项即可
    
    # 元数据配置（可选）
    metadata:
        source: "rewardbench2"            # 数据源标识
        version: "1.0"                    # 数据版本（可选）
        description: "Sample dataset"      # 数据描述（可选）
```

## 🔗 参考资源

### 📚 官方文档
- **Label Studio 官方指南**: https://labelstud.io/guide/
- **Data-Juicer 项目文档**: https://github.com/modelscope/data-juicer
- **HuggingFace Datasets**: https://huggingface.co/docs/datasets/

